First we need to train a model using mlflow as a tracking server and then log its artifact.

// Then we need to create a separate directory and bring in all the Pipfile, Pipfile.lock , predict.py, and test.py file

// We need to modify the predict.py file in order to load the model trained in mlflow

In [70]:
%pip install python-dotenv

In [ ]:
import mlflow
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
import optuna


import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
data = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-11.parquet")


In [ ]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-11-01 00:03:03,2023-11-01 01:04:08,2.0,13.60,1.0,N,132,26,2,61.8,2.75,0.5,0.00,0.00,1.0,66.05,0.0,1.75
1,1,2023-11-01 00:03:28,2023-11-01 00:23:59,0.0,3.50,1.0,N,140,7,1,20.5,3.50,0.5,5.10,0.00,1.0,30.60,2.5,0.00
2,2,2023-10-31 23:58:05,2023-11-01 00:54:03,4.0,18.61,2.0,N,132,230,1,70.0,0.00,0.5,16.54,6.94,1.0,99.23,2.5,1.75
3,2,2023-11-01 00:03:50,2023-11-01 00:04:59,1.0,0.39,1.0,N,236,236,1,4.4,1.00,0.5,1.88,0.00,1.0,11.28,2.5,0.00
4,2,2023-11-01 00:06:30,2023-11-01 00:14:25,1.0,1.20,1.0,N,236,141,1,10.0,1.00,0.5,3.00,0.00,1.0,18.00,2.5,0.00


In [ ]:
data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [ ]:
data.shape[0]*0.8

2671772.0

In [ ]:
def data_preparation(data: pd.DataFrame) -> pd.DataFrame:
    data['duration'] = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']
    data['duration'] = data['duration'].apply(lambda td: td.total_seconds()/60)
    X = data[['passenger_count','trip_distance','PULocationID','DOLocationID','fare_amount']]
    y = data['duration']
    return X,y

In [ ]:
data.shape[0]

3339715

In [ ]:
X_train_val, y_train_val = data_preparation(data[:20000])
X_test, y_test = data_preparation(data[20001:25000])

/tmp/ipykernel_1910/1507299739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['duration'] = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']
/tmp/ipykernel_1910/1507299739.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['duration'] = data['duration'].apply(lambda td: td.total_seconds()/60)
/tmp/ipykernel_1910/1507299739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val, test_size=0.3,random_state=44,shuffle=True)

In [ ]:
xtr = X_train.to_dict(orient='records')
xval = X_val.to_dict(orient='records')

In [ ]:
dv = DictVectorizer()


In [ ]:
train_data = dv.fit_transform(xtr)
val_data = dv.transform(xval)

## Create and optuna hyperparameter tuner


In [ ]:
RandomForestRegressor??

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Source:        
class RandomForestRegressor(ForestRegressor):
    """
    A random forest regressor.

    A random forest is a meta estimator that fits a number of classifying
    decision trees on various sub-samples of the dataset and uses averaging
    to improve the predictive accuracy and control over-fitting.
    The sub-sample size is controlled with the `max_samples` parameter if
    `bootstrap=True` (default), otherwise the whole dataset is used to build
    each tree.

    For a comparison between tree-based ensemble models see the example
   

In [ ]:

def objective(trial):
    param = {
        "n_estimators": trial.suggest_int('n_estimators',100,200,50),
        "max_depth": trial.suggest_int('max_depth',10,50,5),
    }

    rf_reg = RandomForestRegressor(**param)
    rf_reg.fit(X_train,y_train)
    preds = rf_reg.predict(X_val)
    accuracy = mean_squared_error(y_pred=preds, y_true=y_val,squared=False)
    return accuracy


sampler = optuna.samplers.TPESampler()
study = optuna.create_study(direction="minimize",sampler=sampler)

n_trials = 10
study.optimize(objective,n_trials=n_trials)
    

[I 2024-02-10 08:16:02,671] A new study created in memory with name: no-name-773c885e-13ea-4c00-8c23-506e1d05a4dc
/tmp/ipykernel_1910/3927788628.py:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int('n_estimators',100,200,50),
/tmp/ipykernel_1910/3927788628.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "max_depth": trial.suggest_int('max_depth',10,50,5),
[I 2024-02-10 08:16:09,851] Trial 0 finished with value: 42.673368892603236 and parameters: {'n_estimators': 150, 'max_depth': 50}. Best is trial 0 with value: 42.673368892603236.
/tmp/ipykernel_1910/3927788628.py:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int('n_estimators',100,200,50),
/tmp/ipykernel_1910/3927788628.py:4: Future

In [ ]:
best_params = study.best_params

In [ ]:
rf_reg = RandomForestRegressor(**best_params)

In [67]:
mlflow.create_experiment("nyc_taxi_exp_4.2_deploy_1",artifact_location="s3://mlflow-artifacts-remote-11/mlflow_artifacts/")

RestException: BAD_REQUEST: (sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('nyc_taxi_exp_4.2_deploy_1', 's3://mlflow-artifacts-remote-11/mlflow_artifacts/', 'active', 1707553085681, 1707553085681)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [68]:
from mlflow.data.pandas_dataset import PandasDataset
dataset: PandasDataset = mlflow.data.from_pandas(X_train_val[:5])

/home/ubuntu/miniconda3/lib/python3.11/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]


In [69]:
mlflow.set_experiment("nyc_taxi_exp_4.2_deploy_1")
mlflow.set_tracking_uri("http://localhost:5000")

with mlflow.start_run():
    AWS_ACCESS_KEY_ID
    mlflow.log_params(best_params)
    rf_reg.fit(X_train,y_train)
    y_pred = rf_reg.predict(X_val)

    rmse = mean_squared_error(y_val,y_pred,squared=False)
    mlflow.log_metric("rmse",rmse)

    mlflow.log_input(dataset,'train_data')

    mlflow.sklearn.log_model(rf_reg,artifact_path="s3://mlflow-artifacts-remote-11/mlflow_artifacts/")



/home/ubuntu/miniconda3/lib/python3.11/site-packages/mlflow/data/pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)
/home/ubuntu/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.m

NoCredentialsError: Unable to locate credentials